## Test `NetworkModel`

In [1]:
import numpy as np
from coremdlr.models import NetworkModel
from coremdlr.datasets import DepthSequenceGenerator, FaciesDataset

Using TensorFlow backend.


In [2]:
fdset = FaciesDataset(["205-21b-3", "204-20-6a", "204-24a-6","204-20-1Z"],
                    test_wells=["204-19-6"],
                    features=["image"],
                    label_resolution=32)

fdset.load_or_generate_data()

Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('image', (3842, 32, 600, 3))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('image', (3542, 32, 600, 3))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('image', (13006, 32, 600, 3))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('image', (1917, 32, 600, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1884,)), ('top', (1884,)), ('base', (1884,)), ('image', (1884, 32, 600, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [3]:
print(np.bincount(fdset.y_train) / fdset.y_train.size)
print(np.bincount(fdset.y_test) / fdset.y_test.size)

[0.27771552 0.15990496 0.1975613  0.02053167 0.34428655]
[0.49575372 0.16295117 0.15127389 0.         0.19002123]


In [4]:
bcnn_model_args = {
    'feature' : 'image',
    'network' : 'bilinear_cnn',
    'sequence_size' : 12,
    'network_args' : {
        'conv1x1' : 32,
        'backbone_cnn' : 'vgg16',
        'lstm_features' : None,
        'dropout_rate' : 0.25,
        'apply_rowwise' : True
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
        'clipnorm' : 10.
    },
    'loss' : {'ordinal_squared_error': 0.5,
              'categorical_crossentropy': 1.0}
}

bcnn_model = NetworkModel(fdset, model_args=bcnn_model_args)

W0723 15:21:03.097903 140149034301248 deprecation.py:506] From /home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 384, 600, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 12, 18, 512)  14714688    input_2[0][0]                    
__________________________________________________________________________________________________
reduce_A (Conv2D)               (None, 12, 18, 32)   16416       vgg16[1][0]                      
__________________________________________________________________________________________________
reduce_B (Conv2D)               (None, 12, 18, 32)   16416       vgg16[1][0]                      
______________________________________________________________________________________________

In [5]:
fit_args = {
    'batch_size' : 4,
    'step_size' : bcnn_model.sequence_size - 2,
    'epochs' : 3,
}

bcnn_model.fit(fdset, **fit_args)

Shapes of `(batch_X, batch_y)`: (4, 384, 600, 3), (4, 12, 5)


W0723 15:21:11.933040 140149034301248 deprecation.py:323] From /home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
557/557 [==============================] - 92s 166ms/step - loss: 2.3525 - acc: 0.5198 - val_loss: 3.0643 - val_acc: 0.4978
Epoch 2/3
557/557 [==============================] - 86s 154ms/step - loss: 2.8277 - acc: 0.3382 - val_loss: 2.6483 - val_acc: 0.1906
Epoch 3/3
557/557 [==============================] - 85s 152ms/step - loss: 4.5499 - acc: 0.3448 - val_loss: 8.2923 - val_acc: 0.4978


2.648347092435715

In [6]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(deepten_model.network).create(prog='dot', format='svg'))

NameError: name 'deepten_model' is not defined

In [ ]:
strip = fdset.wells[0]._make_striplog()

In [ ]:
import pandas as pd
from io import StringIO

pd.read_csv(StringIO(strip.to_csv()))

In [ ]:
from striplog import Decor, Component, Legend

###++++++++++++++++++++###
### Facies Definitions ###
###++++++++++++++++++++###

facies = {
    'nc': Component({'lithology': 'none', 'train':'n'}),
    'bs': Component({'lithology': 'sandstone', 'train': 'n'}),
    's': Component({'lithology': 'sandstone', 'train':'y'}),
    'is': Component({'lithology': 'clay-prone sandstone', 'train':'y'}),
    'ih': Component({'lithology': 'sandy mudstone', 'train':'y'}),
    'sh': Component({'lithology': 'mudstone', 'train':'y'}),
    't': Component({'lithology': 'turbidite', 'train':'y'}),
}

lithologies = [c.lithology for c in facies.values()]

def lithology_to_key(lithology_str):
    try:
        litho = lithology_str.split(',')[0].lower()
        idx = lithologies.index(litho)
        return list(facies.keys())[idx]
    except IndexError:
        raise ValueError('{} is not present in lithologies: {}'.format(lithology_str, lithologies))

In [ ]:
df = pd.read_csv(StringIO(strip.to_csv()))
df.Component = df.Component.apply(lithology_to_key)
df

In [ ]:
df.columns = ['top', 'base', 'lithology']
df.to_csv()

In [ ]:
from coremdlr.datasets import strip_config

In [ ]:
list(strip_config.values())[0].lithology

In [ ]:
fdset.wells[0]._y.size

In [ ]:
fdset.wells[0]._image.max()

In [ ]:
fdset.wells[0].data_path

In [ ]:
img = np.load('/home/administrator/Dropbox/core_data/facies/train_data/204-19-6_image.npy')
img.min(), img.max()

In [ ]:
a = np.random.rand(10,5)
a

In [ ]:
a[np.where(a > 0.5)]

In [ ]:
fdset.X_train['pseudoGR'].shape